# SQLite3 - Python

Code vu sur le diaporama.

- Création de la base de données, 

- Création de table `utilisateurs`, 

- Insertion de données, 

- Modification de connées, 

- Effacement d'entrées, 

- Sélection de données.

Exécutez ce code pour générer la base données avec une table utilisateurs.

In [1]:
import sqlite3

# Effacer le fichier de base de données s'il existe déjà
import os
if os.path.exists("database.db"):
    os.remove("database.db")

# Création / ouverture d'une base de données SQLite3
with sqlite3.connect('database.db') as conn:

    # écriture dans la base de données
    cur = conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS utilisateurs (id INTEGER PRIMARY KEY AUTOINCREMENT, nom TEXT, email TEXT)")

    # ajouter des données dans la base de données
    cur.execute("INSERT INTO utilisateurs (nom, email) VALUES (?, ?)", ("Alice", "alice@ex.com"))

    # Insertion de données dans la table 'utilisateurs'
    nouvel_utilisateur = ("John Doe", "john.doe@example.com")
    cur.execute("INSERT INTO utilisateurs (nom, email) VALUES (?, ?)", nouvel_utilisateur)

    # Pour insérer plusieurs lignes en une seule requête, utilisez executemany
    nouveaux_utilisateurs = [("Jane Smith", "jane.smith@example.com"), ("Bob Johnson", "bob.johnson@example.com")]
    cur.executemany("INSERT INTO utilisateurs (nom, email) VALUES (?, ?)", nouveaux_utilisateurs)

    # Validez la transaction et enregistrez les modifications
    conn.commit()

    # Lire les données de la base de données
    cur.execute("SELECT * FROM utilisateurs")
    resultat = cur.fetchall()

    # Afficher les données
    print(" Après l'insertion des données : ".center(80, "-"))
    for ligne in resultat:
        print(ligne)

    # Mise à jour du champ 'email' pour un utilisateur spécifique
    utilisateur_id = 1
    nouvel_email = "nouveau.email.alice@example.com"
    cur.execute("UPDATE utilisateurs SET email = ? WHERE id = ?", (nouvel_email, utilisateur_id))

    # stopper la transaction et enregistrer les modifications dans la base de données
    conn.commit()

    # Suppression d'un utilisateur spécifique en fonction de son ID
    utilisateur_id = 4
    cur.execute("DELETE FROM utilisateurs WHERE id = ?", (utilisateur_id,))

    # stopper la transaction et enregistrer les modifications dans la base de données
    conn.commit()

    # Lire les données de la base de données
    cur.execute("SELECT * FROM utilisateurs")
    resultat = cur.fetchall()

    # Afficher les données
    print(" Après la mise à jour et la suppression des données : ".center(80, "-"))
    for ligne in resultat:
        print(ligne)

# fermer la connexion
# conn.close() non nécessaire car le 'with' s'en charge

----------------------- Après l'insertion des données : ------------------------
(1, 'Alice', 'alice@ex.com')
(2, 'John Doe', 'john.doe@example.com')
(3, 'Jane Smith', 'jane.smith@example.com')
(4, 'Bob Johnson', 'bob.johnson@example.com')
------------- Après la mise à jour et la suppression des données : -------------
(1, 'Alice', 'nouveau.email.alice@example.com')
(2, 'John Doe', 'john.doe@example.com')
(3, 'Jane Smith', 'jane.smith@example.com')


## Exercice 1 - Table `livres`

Ajoutez une table `livres` et une table `auteurs` à la base de données.

Ces tables contiendront respectivement les colonnes suivantes :

1. Livres :

    - `id` : identifiant integer clé primaire (autoincrementée).
    - `titre` : texte
    - `pitch` : texte (résumé)
    - `auteur_id` : clé étrangère vers la table `auteurs`
    - `date_public` : date du livre

.

2. Auteurs :

    - `id` : identifiant integer clé primaire (autoincrementée).
    - `nom_auteur` : nom complet de l'auteur


Vous devez faire le script python pour lire le fichier JSON (ou un autre si besoin) et automatiquement générer les deux tables citées.

Cette procédure automatique doit pouvoir aussi être appelée si on donne un fichier JSON pour `ajouter` des données à la base de données. On considérera que dans ce cas le nom d'un auteur est un identifiant unique (i.e. s'il est déjà dans la table, on n'ajoute pas un nouvel auteur), et de même que le titre d'un livre est unique.

Pour contrôler la bonne exécution du code, pensez à faire quelques affichages voire à générer un fichier log.txt :

```
nouvel auteur Victor Hugo ajouté id = 1
nouvel auteur Gustave Flaubert ajouté id = 2
nouvel auteur Émile Zola ajouté id = 3
nouvel auteur Albert Camus ajouté id = 4
auteur Albert Camus déjà existant
nouvel auteur Charles Baudelaire ajouté id = 5
nouvel auteur Pierre Choderlos de Laclos ajouté id = 6
nouvel auteur Louis-Ferdinand Céline ajouté id = 7
nouvel auteur Georges Perec ajouté id = 8
nouvel auteur Khaled Hosseini ajouté id = 9
nouvel auteur Umberto Eco ajouté id = 10
nouvel auteur Franz Kafka ajouté id = 11
auteur Émile Zola déjà existant
nouvel auteur Alain Damasio ajouté id = 12
nouvel auteur Maurice Druon ajouté id = 13
nouvel auteur Alexandre Dumas ajouté id = 14
nouvel auteur Léon Tolstoï ajouté id = 15
auteur Alexandre Dumas déjà existant
nouvel auteur Patrick Süskind ajouté id = 16
auteur Georges Perec déjà existant
nouvel auteur Stendhal ajouté id = 17
nouvel auteur Guy de Maupassant ajouté id = 18
auteur Émile Zola déjà existant
nouvel auteur Alain-Fournier ajouté id = 19
auteur Gustave Flaubert déjà existant
```


In [2]:
from ast import Invert
import json
import sqlite3
import logging

#Création de la base de donnée
def create_tables(conn):
    """Crée les tables livres et auteurs."""
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS auteurs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nom_auteur TEXT UNIQUE
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS livres (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            titre TEXT UNIQUE,
            pitch TEXT,
            auteur_id INTEGER,
            date_public DATE,
            FOREIGN KEY(auteur_id) REFERENCES auteurs(id)
        )
    ''')
    conn.commit()

donnee_livre =""
donnee_emprunt =""
liste_affiche = []

#Programme
with open("data_books.json", "r") as f:
    donnee_livre = json.load(f)

with sqlite3.connect('database.db') as conn:
    create_tables(conn)
    cur = conn.cursor()
    print("\n\nCréation table auteurs et vérification auteur")
    liste_auteur = []
    for i in donnee_livre:
        id = i.get('id')
        title = i.get('title')
        content = i.get('content')
        author = i.get('author')
        date = i.get('date')
        if author in liste_auteur:
            print("Auteur " + author + "  déjà existant")
            liste_affiche.append("Auteur " + author + "  déjà existant")
        elif author not in liste_auteur:
            cur.execute("INSERT INTO auteurs (nom_auteur) VALUES (?)", (author,))
            conn.commit()
            cur.execute("SELECT id FROM auteurs WHERE nom_auteur = ?", (author,))
            for resultat in cur.fetchall() :
                print("nouvel auteur " + author + " ajouté id = " + str(resultat[0]))
            liste_affiche.append("nouvel auteur " + author + " ajouté id = " + str(resultat[0]))
            liste_auteur.append(author)
        else:
            print("!!!Erreur!!!")
    print("\n\n\nCréation de la table livres")
    cur.execute("SELECT id FROM auteurs WHERE nom_auteur = ?", (author,))
    for id_auteur in cur.fetchall() :
        id_auteur = str(id_auteur[0])
        cur.execute("INSERT INTO livres (titre, pitch, auteur_id, date_public) VALUES (?,?,?,?)", (title, content, id_auteur, date,))
        conn.commit()
    


for i in liste_affiche:
    print(i)



Création table auteurs et vérification auteur
nouvel auteur Victor Hugo ajouté id = 1
nouvel auteur Gustave Flaubert ajouté id = 2
nouvel auteur Émile Zola ajouté id = 3
nouvel auteur Albert Camus ajouté id = 4
Auteur Albert Camus  déjà existant
nouvel auteur Charles Baudelaire ajouté id = 5
nouvel auteur Pierre Choderlos de Laclos ajouté id = 6
nouvel auteur Louis-Ferdinand Céline ajouté id = 7
nouvel auteur Georges Perec ajouté id = 8
nouvel auteur Khaled Hosseini ajouté id = 9
nouvel auteur Umberto Eco ajouté id = 10
nouvel auteur Franz Kafka ajouté id = 11
Auteur Émile Zola  déjà existant
nouvel auteur Alain Damasio ajouté id = 12
nouvel auteur Maurice Druon ajouté id = 13
nouvel auteur Alexandre Dumas ajouté id = 14
nouvel auteur Léon Tolstoï ajouté id = 15
Auteur Alexandre Dumas  déjà existant
nouvel auteur Patrick Süskind ajouté id = 16
Auteur Georges Perec  déjà existant
nouvel auteur Stendhal ajouté id = 17
nouvel auteur Guy de Maupassant ajouté id = 18
Auteur Émile Zola  dé

## Exercice 2 - Ajouter colonne `emprunteur_id`

Ajouter une colonne `emprunteur_id` à la table `livres`. Ce sera une clé étrangère reliée à la table `utilisateurs`.

Voir les ordres `SQL` sur la doc : [https://www.sqlitetutorial.net/sqlite-cheat-sheet/](https://www.sqlitetutorial.net/sqlite-cheat-sheet/)

Générez de 1 à 4 livres empruntés (au hasard) pour chaque utilisateur.

Pensez à générer des affichages ou un log.txt pour vérifier l'exécution de vos scripts.

```
livre La Peste emprunté par Alice
livre L'Assommoir emprunté par John Doe
livre La Métamorphose emprunté par John Doe
livre Germinal emprunté par John Doe
livre Les Trois Mousquetaires emprunté par John Doe
livre La Guerre et la Paix emprunté par Jane Smith
livre Les Rois maudits emprunté par Jane Smith
livre Le Rouge et le Noir emprunté par Jane Smith
```


In [6]:
# ajout de colonne `emprunteur_id` à la table `livres`. Ce sera une clé étrangère reliée à la table `utilisateurs`.

# Votre code ici...
cur.execute("ALTER TABLE livres ADD COLUMN emprunteur_id INTEGER")
conn.commit()


In [22]:
# ajout d'emprunts au hasard (de 1 à 4 par utilisateur)

# Votre code ici...
from random import randint
import sqlite3


def rand14():
    fois = randint(1,4)
    chiffre = []
    for i in range(fois):
        while True:
            nombre=randint(1,24)
            if nombre not in chiffre:
                chiffre.append(nombre)
                break
    return chiffre

liste_nom = ("Alice", "John Doe", "Jane Smith")

with sqlite3.connect('database.db') as conn:
    cur = conn.cursor()
    for nom in liste_nom:
        liste_nombre = rand14()
        for nombre in liste_nombre:
            cur.execute("SELECT titre FROM livres WHERE id = ?", (nombre,))
            for titre in cur.fetchall() :
                cur.execute("SELECT id FROM utilisateurs WHERE nom = ?", (nom,))
                for id_utilisateurs in cur.fetchall() :
                    cur.execute("UPDATE livres SET emprunteur_id = ? WHERE id = ?", (id_utilisateurs[0], nombre))
                    print(titre[0] + " a été emprunté par " + nom)


La Guerre et la Paix a été emprunté par Alice
Germinal a été emprunté par Alice
Le Nom de la rose a été emprunté par Alice
Bel-Ami a été emprunté par Alice
La Vie mode d'emploi a été emprunté par John Doe
Madame Bovary a été emprunté par John Doe
Le Nom de la rose a été emprunté par John Doe
Madame Bovary a été emprunté par Jane Smith
Les Misérables a été emprunté par Jane Smith
Germinal a été emprunté par Jane Smith
Le Comte de Monte-Cristo a été emprunté par Jane Smith


## Exercice 3 - Génération d'API

La base de données est finie !

Maintenant on souhaite créer en Flask / Jinja2 une API ReST pour cette table.

Faites l'API avec les "endpoints" suivants :

1. Chemins `/utilisateurs`, `/livres`, `/auteurs` : renvoient la liste en JSON des tables complètes

1. Chemin `/utilisateur/<utilisateur>` : renvoie le dictionnaire correspondant à l'utilisateur d'id `utilisateur` ou par l'utilisateur de nom `utilisateur` si un seul utilisateur porte ce nom-là. Si plusieurs portent le même nom, une erreur est renvoyée.

1. Chemin `/utilisateur/emprunts/<utilisateur>` : renvoie la liste des livres empruntés par l'utilisateur d'id `utilisateur` ou par l'utilisateur de nom `utilisateur` si un seul utilisateur porte ce nom-là.

1. Chemin `/livres/siecle/<numero>` : renvoie la liste des livres du siècle marqué.

1. Chemin `/livres/ajouter` : en POST ajoute un livre au format identique au fichier JSON (si l'auteur n'existe pas encore il est ajouté)

1. Chemin `/utilisateur/ajouter` : en POST ajoute un utilisateur (format {"nom": nom_user, "email": email_user})

1. Chemin `/utilisateur/<utilisateur>/supprimer` : en DELETE

1. Chemin `/utilisateur/{utilisateur_id}/emprunter/{livre_id}` : en PUT, permet d'emprunter un livre

1. Chemin `/utilisateur/{utilisateur_id}/rendre/{livre_id}` : en PUT, permet de rendre un livre


In [ ]:
"""
Faites l'API avec les "endpoints" suivants :

1. Chemins `/utilisateurs`, `/livres`, `/auteurs` : renvoient la liste en JSON des tables complètes
1. Chemin `/utilisateur/<utilisateur>` : renvoie le dictionnaire correspondant à l'utilisateur d'id `utilisateur` ou par l'utilisateur de nom `utilisateur` si un seul utilisateur porte ce nom-là. Si plusieurs portent le même nom, une erreur est renvoyée.
1. Chemin `/utilisateur/emprunts/<utilisateur>` : renvoie la liste des livres empruntés par l'utilisateur d'id `utilisateur` ou par l'utilisateur de nom `utilisateur` si un seul utilisateur porte ce nom-là.
1. Chemin `/livres/siecle/<numero>` : renvoie la liste des livres du siècle marqué.
1. Chemin `/livres/ajouter` : en POST ajoute un livre au format identique au fichier JSON (si l'auteur n'existe pas encore il est ajouté)
1. Chemin `/utilisateur/ajouter` : en POST ajoute un utilisateur (format {"nom": nom_user, "email": email_user})
1. Chemin `/utilisateur/<utilisateur>/supprimer` : en DELETE
1. Chemin `/utilisateur/{utilisateur_id}/emprunter/{livre_id}` : en PUT, permet d'emprunter un livre
1. Chemin `/utilisateur/{utilisateur_id}/rendre/{livre_id}` : en PUT, permet de rendre un livre
"""
# Votre code ici...


## Exercice 4 - Swagger (OpenAPI documentation)

Ecrivez un fichier "Swagger" pour l'API : "openapi.json"

Ecrivez les deux premiers "endpoints" de l'API dans un fichier compatible OpenAPI 3.1


In [ ]:
# Ou bien pour génération automatique : Code en FastAPI

import sqlite3
from fastapi import FastAPI, HTTPException, Path, Body
from typing import Union, Callable, Annotated
from pydantic import BaseModel


# définition des classes pour les modèles

class Utilisateur(BaseModel):
    id: int
    nom: str
    email: str
    def __init__(self, id, nom, email):
        super().__init__(id=id, nom=nom, email=email)

# Votre code ici...


# .....................................................................





import uvicorn
if __name__ == '__main__':
    # uvicorn.run(app, host='localhost', port=5009)
    # code pour lancer depuis un notebook Jupyter
    import nest_asyncio
    nest_asyncio.apply()
    uvicorn.run(app, host='localhost', port=5009)